### Integrating  Google reCaptcha:

Integrating Google reCAPTCHA into your Django application involves several steps, including signing up for reCAPTCHA, modifying your forms, views, and templates, and adding server-side validation. Here's how to do it:<br>

#### Step 1: Sign Up for Google reCAPTCHA
1. Go to the <a href='https://www.google.com/recaptcha/about/'>Google reCAPTCHA site</a>.
2. Register your site and get the site key and secret key.
    - reCaptcha type should be Challenge(v2) -> "I'm not a robot" Checkbox
    - since we are going to use it on the development server use 127.0.0.1 for the domain

#### Step 2: Install Required Packages
You'll need the django-recaptcha package. Install it via pip:

In [ ]:
pip install django-recaptcha

#### Step 3: Update Your Django Settings

Add your reCAPTCHA keys to your settings.py:
- Use environment variables, don't put them in your code

In [ ]:
from os import environ

RECAPTCHA_PUBLIC_KEY = environ.get('recaptcha-site-key')
RECAPTCHA_PRIVATE_KEY = environ.get('recaptcha-secret-key')

#### Step 4: Modify Your Form

Update your CustomUserCreationForm to include the reCAPTCHA field:

In [ ]:
from django_recaptcha.fields import ReCaptchaField


class CustomUserCreationForm(UserCreationForm):
    age = forms.IntegerField(required=False)
    captcha = ReCaptchaField()

    class Meta:
        model = CustomUser
        fields = ('username', 'age', 'email', 'captcha')

#### Step 5: Update Your Template

Add the reCAPTCHA widget to your template:

In [ ]:
{% block content %}
<div class="main-wrapper">
    <form action="" method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Register</button>
    </form>
</div>s
<script src="https://www.google.com/recaptcha/api.js" async defer></script>
{% endblock content %}

#### Step 6: Server-Side Validation

The django-recaptcha package will handle server-side validation automatically when you use ReCaptchaField. Ensure you catch validation errors appropriately.

Here's the complete view with error handling:

In [ ]:
class SignUpView(CreateView):
    form_class = CustomUserCreationForm
    template_name = 'registration/signup.html'
    success_url = reverse_lazy('login')

    def form_valid(self, form):
        response = super().form_valid(form)
        # Any additional logic can be added here
        return response

    def form_invalid(self, form):
        response = super().form_invalid(form)
        # Handle form errors, including reCAPTCHA errors
        return response

#### Step 7: Add reCAPTCHA Settings to Template Context

If RECAPTCHA_PUBLIC_KEY is not already available in your templates, add it in your views or context processors.

In your views:

In [ ]:
from django.conf import settings

class SignUpView(CreateView):
    form_class = CustomUserCreationForm
    template_name = 'registration/signup.html'
    success_url = reverse_lazy('login')

    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['RECAPTCHA_PUBLIC_KEY'] = settings.RECAPTCHA_PUBLIC_KEY
        return context

    def form_valid(self, form):
        response = super().form_valid(form)
        # Any additional logic can be added here
        return response

    def form_invalid(self, form):
        response = super().form_invalid(form)
        # Handle form errors, including reCAPTCHA errors
        return response
        

Step 8: Adding 'django_recaptcha' to INSTALLED_APPS:

Navigate to your setting.py file and add 'django_recaptcha' to the INSATALLED_APPS list


NOW, your captcha should work just fine!


There is one problem though, the captcha field is being shown after email and before password1 and password 2!

To fix this you need to manualy set password1 and password2 before captcha in your form.py file:

In [ ]:
class CustomUserCreationForm(UserCreationForm):
    age = forms.IntegerField(required=False)
    captcha = ReCaptchaField()

    class Meta:
        model = CustomUser
        fields = ('username', 'age', 'email', 'password1', 'password2', 'captcha', )

### Question: How are we supposed to add reCaptcha to our login page?

Do you realize the problem? the login.html template is using a prebuild view offered by django in the auth app and we are not supposed to modify auth.views.py file.

So, what should we do?

Well, unfortunetly we are not able to achieve this without modifying auth.views.py<br>

Don't worry the modification is only one line of code and does not effect anything else.

1. Navigate to auth.views.py
2. import the following:

In [ ]:
from django_recaptcha.fields import ReCaptchaField
from django_recaptcha.widgets import ReCaptchaV2Checkbox

3. Located 'LoginView' class
4. look for 'get_form_class' method and modify it:

In [ ]:
def get_form_class(self):
    # Check if the user is accessing the admin login page
    if 'admin' in self.request.path:
        return self.authentication_form or self.form_class  # Use the default authentication form without reCAPTCHA
    else:
        # Add the reCAPTCHA field to the form class
        self.form_class.base_fields['captcha'] = ReCaptchaField(widget=ReCaptchaV2Checkbox)
        return self.authentication_form or self.form_class

Please undertsand that the above code will make sure that the captcha is not being required on the admin login page.<br>

5. Modify login.html template:

In [ ]:
{% block content %}
<div class="main-wrapper">
    <form action="" method='POST'>
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Login</button>
    </form>
        <a href="{% url "password_reset" %}">Forgot Password?</a>
</div>
<script src="https://www.google.com/recaptcha/api.js" async defer></script>
{% endblock content %}

You may need to add some css code to style the captcha field.


I understand that styling froms might be exhusting! so, let me teach you a better way.


Did you know that you could use a package to add bootstrap form design to DTL (django Template Language)?

1. Install django-crispy-forms using pip

In [ ]:
python -m pip install crispy-bootstrap4

2. Add 'crispy_forms' and 'crispy_bootstrap4' to INSTALLED_APPS inside settings.py

In [ ]:
'crispy_forms',
'crispy_bootstrap4'

3. Add the following constant to the settings.py:

In [ ]:
CRISPY_ALLOWED_TEMPLATE_PACKS = "bootstrap4"
CRISPY_TEMPLATE_PACK = "bootstrap4"

4. use the following at the top of your template and after loading static:

In [ ]:
{% load crispy_forms_tags %}

5. Add the following lines to your css_files block:

In [ ]:
 <!-- Latest compiled and minified CSS -->
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.6.2/dist/css/bootstrap.min.css">
 <!-- jQuery library -->
 <script src="https://cdn.jsdelivr.net/npm/jquery@3.7.1/dist/jquery.slim.min.js"></script>
 <!-- Popper JS -->
 <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.1/dist/umd/popper.min.js"></script>
 <!-- Latest compiled JavaScript -->
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.6.2/dist/js/bootstrap.bundle.min.js"></script> 

6. Now you can use {{ form|crispy }} to have your forms styled automatically.

<span style='color:red;'>NOTE:</span> You need to delete all the css and redesign your page using bootstrap now.<br><br>
<span style='color:red;'>NOTE:</span> Please undertsand that bootstrap crispy filter does not go well with reCaptcha.

#### End of chapter 13